In [1]:
import os, sys
import cv2
import time
import numpy as np
import torch

In [2]:
def load_avi():
    actions = ['ㄱ', 'ㄴ', 'ㄷ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅅ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ',
                'ㅏ', 'ㅑ', 'ㅓ', 'ㅕ', 'ㅗ', 'ㅛ', 'ㅜ', 'ㅠ', 'ㅡ', 'ㅣ',
                'ㅐ', 'ㅒ', 'ㅔ', 'ㅖ', 'ㅢ', 'ㅚ', 'ㅟ']
    label_dict = dict()

    for i in range(len(actions)):
        label_dict[actions[i]] = i

    videoFolderPath = "./SignLanguagedataset/output_video"
    videoTestList = os.listdir(videoFolderPath)

    testTargetList =[]

    for videoPath in videoTestList:
        actionVideoPath = f'{videoFolderPath}/{videoPath}'
        actionVideoList = os.listdir(actionVideoPath)
        for actionVideo in actionVideoList:
            fullVideoPath = f'{actionVideoPath}/{actionVideo}'
            testTargetList.append(fullVideoPath)

    testTargetList = sorted(testTargetList, key=lambda x:x[x.find("/", 9)+1], reverse=True)

    return testTargetList, label_dict

video_paths, label_dict = load_avi()
print(f"testTargetList : 영상개수={len(video_paths)}") 
print("label_dict : ", label_dict)
label_dict

testTargetList : 영상개수=363
label_dict :  {'ㄱ': 0, 'ㄴ': 1, 'ㄷ': 2, 'ㄹ': 3, 'ㅁ': 4, 'ㅂ': 5, 'ㅅ': 6, 'ㅇ': 7, 'ㅈ': 8, 'ㅊ': 9, 'ㅋ': 10, 'ㅌ': 11, 'ㅍ': 12, 'ㅎ': 13, 'ㅏ': 14, 'ㅑ': 15, 'ㅓ': 16, 'ㅕ': 17, 'ㅗ': 18, 'ㅛ': 19, 'ㅜ': 20, 'ㅠ': 21, 'ㅡ': 22, 'ㅣ': 23, 'ㅐ': 24, 'ㅒ': 25, 'ㅔ': 26, 'ㅖ': 27, 'ㅢ': 28, 'ㅚ': 29, 'ㅟ': 30}


{'ㄱ': 0,
 'ㄴ': 1,
 'ㄷ': 2,
 'ㄹ': 3,
 'ㅁ': 4,
 'ㅂ': 5,
 'ㅅ': 6,
 'ㅇ': 7,
 'ㅈ': 8,
 'ㅊ': 9,
 'ㅋ': 10,
 'ㅌ': 11,
 'ㅍ': 12,
 'ㅎ': 13,
 'ㅏ': 14,
 'ㅑ': 15,
 'ㅓ': 16,
 'ㅕ': 17,
 'ㅗ': 18,
 'ㅛ': 19,
 'ㅜ': 20,
 'ㅠ': 21,
 'ㅡ': 22,
 'ㅣ': 23,
 'ㅐ': 24,
 'ㅒ': 25,
 'ㅔ': 26,
 'ㅖ': 27,
 'ㅢ': 28,
 'ㅚ': 29,
 'ㅟ': 30}

In [3]:
from tqdm import tqdm
import re

def make_directory(output_dir, data_folder):
    folder = f"{output_dir}/{data_folder}"
    image_folder = f"{output_dir}/{data_folder}/images"
    labels_folder = f"{output_dir}/{data_folder}/labels"

    if not os.path.exists(folder):
        os.makedirs(image_folder)
        os.makedirs(labels_folder)

# Example usage
output_dir = './dataset'
make_directory(output_dir, data_folder="train")
make_directory(output_dir, data_folder="valid")
make_directory(output_dir, data_folder="test")

In [4]:
image_folder = "./dataset/train/images"
labels_folder = "./dataset/train/labels"

image_files = set(os.listdir(image_folder))
label_files = set(os.listdir(labels_folder))

# 이미지 파일에서 레이블을 제거하여 순수한 파일 이름을 얻음
image_base_names = {os.path.splitext(name)[0] for name in image_files}
label_base_names = {os.path.splitext(name)[0] for name in label_files}

# 불일치 찾기
missing_labels = image_base_names - label_base_names
missing_images = label_base_names - image_base_names
print("train : ",len(missing_labels), len(label_base_names))

train :  12775 52281


In [5]:
image_folder = "./dataset/valid/images"
labels_folder = "./dataset/valid/labels"

image_files = set(os.listdir(image_folder))
label_files = set(os.listdir(labels_folder))

# 이미지 파일에서 레이블을 제거하여 순수한 파일 이름을 얻음
image_base_names = {os.path.splitext(name)[0] for name in image_files}
label_base_names = {os.path.splitext(name)[0] for name in label_files}

# 불일치 찾기
missing_labels = image_base_names - label_base_names
missing_images = label_base_names - image_base_names
print("valid : ",len(missing_labels), len(label_base_names))

valid :  1635 6497


In [6]:
image_folder = "./dataset/test/images"
labels_folder = "./dataset/test/labels"

image_files = set(os.listdir(image_folder))
label_files = set(os.listdir(labels_folder))

# 이미지 파일에서 레이블을 제거하여 순수한 파일 이름을 얻음
image_base_names = {os.path.splitext(name)[0] for name in image_files}
label_base_names = {os.path.splitext(name)[0] for name in label_files}

# 불일치 찾기
missing_labels = image_base_names - label_base_names
missing_images = label_base_names - image_base_names

print("test : ", len(missing_labels), len(label_base_names))

test :  1569 6570


In [7]:
import os

def create_file_list(image_dir, label_dir, output_file):
    with open(output_file, 'w') as file:
        for image_filename in os.listdir(image_dir):
            if image_filename.endswith('.jpg') or image_filename.endswith('.jpeg') or image_filename.endswith('.png'):
                label_filename = image_filename.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt')
                label_path = os.path.join(label_dir, label_filename)
                if os.path.exists(label_path):
                    image_path = os.path.join(image_dir, image_filename)
                    file.write(image_path + '\n')
                    
# 각 디렉토리 경로 설정
train_images_dir = 'dataset/train/images'
train_labels_dir = 'dataset/train/labels'
valid_images_dir = 'dataset/valid/images'
valid_labels_dir = 'dataset/valid/labels'
test_images_dir = 'dataset/test/images'
test_labels_dir = 'dataset/test/labels'

# 출력 파일 경로 설정
train_list_file = 'dataset/train/train.txt'
valid_list_file = 'dataset/valid/valid.txt'
test_list_file = 'dataset/test/test.txt'

# 파일 리스트 생성
create_file_list(train_images_dir, train_labels_dir, train_list_file)
create_file_list(valid_images_dir, valid_labels_dir, valid_list_file)
create_file_list(test_images_dir, test_labels_dir, test_list_file)

In [3]:
import os
import shutil

def copy_files(file_list, target_dir):
    with open(file_list, 'r') as file:
        for line in file:
            image_path = line.strip()
            label_path = image_path.replace('/images/', '/labels/').replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt')

            if os.path.exists(image_path) and os.path.exists(label_path):
                # 이미지 파일을 대상 디렉토리로 복사
                shutil.copy(image_path, target_dir)
                # 레이블 파일을 대상 디렉토리로 복사
                shutil.copy(label_path, target_dir)

# 각 파일 리스트 경로 설정
train_list_file = 'dataset/train/train.txt'
valid_list_file = 'dataset/valid/valid.txt'
test_list_file = 'dataset/test/test.txt'

# 복사할 대상 디렉토리 설정
merged_dir = 'dataset/merged'
os.makedirs(merged_dir, exist_ok=True)

# 파일 복사
copy_files(train_list_file, merged_dir)
copy_files(valid_list_file, merged_dir)
copy_files(test_list_file, merged_dir)

In [4]:
import os

image_folder = "./dataset/test/images"
labels_folder = "./dataset/test/labels"
# 폴더가 이미 존재하는 경우
image_files = os.listdir(image_folder)
label_files = os.listdir(labels_folder)
print(f"image({len(image_files)}), label({len(label_files)})")


image(1968), label(1659)
